In [ ]:
# use dream_proj conda environment
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
# auto reload source files
%load_ext autoreload
%autoreload 2
# make docstrings from my own functions available
#%config IPCompleter.use_jedi = True
# add source directory to path
source_path = os.path.abspath(os.path.join('../..'))
if source_path not in sys.path:
    sys.path.append(os.path.join(source_path, 'source'))
# read source files
import process_tcgabiolinks

In [5]:
sample_sheet = pd.read_csv(
    "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample_sheet_no_duplicate_rows.tsv",
    sep = "\t"
    )
clinical = pd.read_csv(
    "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/clinical.tsv",
    sep = "\t"
    )

followup = pd.read_csv(
    "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/follow_up.tsv",
    sep = "\t"
    )
biospecimen_sample = pd.read_csv(
    "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample.tsv",
    sep = "\t"
    )
"""
sample_sheet['first_sample_id'] = sample_sheet['Sample ID'].str.split(",", expand = True)[0]
sample_sheet['first_sample_type'] = sample_sheet['Sample Type'].str.split(", ", expand = True)[0]
sample_sheet['first_case_id'] = sample_sheet['Case ID'].str.split(", ", expand = True)[0]
sample_sheet.to_csv("/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample_sheet.tsv", sep = "\t", index = False)
# drop rows that are transcriptome and the file name does not contain rna_seq
sample_sheet = sample_sheet.query(
    '((`Data Category` != "Transcriptome Profiling") | (`File Name`.str.contains("rna_seq") == True))'
    )
# drop duplicate rows of sample_sheet
# not considering 'File ID', 'File Name', 'Sample ID',
rows_to_consider = ['Data Category', 'Data Type', 'Project ID', 'Case ID', 
       'Sample Type', 'first_sample_id', 'first_sample_type', 'first_case_id']
sample_sheet = sample_sheet.drop_duplicates(
    subset = rows_to_consider, keep = 'first'
)
# Drop mutation rows where first_sample_id and first_case_id are duplicated
mut_rows_to_keep = sample_sheet.query(
    "`Data Category` == 'Simple Nucleotide Variation'"
    ).drop_duplicates(subset = ['first_case_id', 'first_sample_id'])
sample_sheet = sample_sheet.query("`Data Category` != 'Simple Nucleotide Variation'")
# add back mutation rows
sample_sheet = pd.concat([sample_sheet, mut_rows_to_keep])
sample_sheet.to_csv("/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample_sheet_no_duplicate_rows.tsv", sep = "\t", index = False)

"""


'\nsample_sheet[\'first_sample_id\'] = sample_sheet[\'Sample ID\'].str.split(",", expand = True)[0]\nsample_sheet[\'first_sample_type\'] = sample_sheet[\'Sample Type\'].str.split(", ", expand = True)[0]\nsample_sheet[\'first_case_id\'] = sample_sheet[\'Case ID\'].str.split(", ", expand = True)[0]\nsample_sheet.to_csv("/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample_sheet.tsv", sep = "\t", index = False)\n# drop rows that are transcriptome and the file name does not contain rna_seq\nsample_sheet = sample_sheet.query(\n    \'((`Data Category` != "Transcriptome Profiling") | (`File Name`.str.contains("rna_seq") == True))\'\n    )\n# drop duplicate rows of sample_sheet\n# not considering \'File ID\', \'File Name\', \'Sample ID\',\nrows_to_consider = [\'Data Category\', \'Data Type\', \'Project ID\', \'Case ID\', \n       \'Sample Type\', \'first_sample_id\', \'first_sample_type\', \'first_case_id\']\nsample_sheet = sample_sheet.drop_duplicates(\n    subset = rows_to_cons

# Mutation

In [245]:
# all mutation files are uniquely identified by first_sample_id
sample_sheet.query("`Data Category` == 'Simple Nucleotide Variation'")[ 'first_sample_id'].value_counts()

first_sample_id
C3L-00017-31    1
C3N-00831-31    1
C3L-01967-31    1
C3N-02266-01    1
C3L-02389-01    1
               ..
C3L-02515-31    1
C3N-02694-31    1
C3L-00156-02    1
C3L-01106-02    1
C3L-02164-03    1
Name: count, Length: 1277, dtype: int64

In [246]:
cptac_mutation = process_tcgabiolinks.main(
    path = "/cellar/users/zkoch/dream/data/tcga/GDCdata/CPTAC-3/harmonized/Simple_Nucleotide_Variation/Masked_Somatic_Mutation",
    dataset_name = 'CPTAC-3',
    output_path = "/cellar/users/zkoch/dream/data/tcga/processed_data",
    datatype = 'mutation',
    sample_sheet_fn = "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample_sheet_no_duplicate_rows.tsv"
)

Read 1448 files


/cellar/users/zkoch/dream/source/process_tcgabiolinks.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs, axis=0).reset_index(drop = True)


Saved to /cellar/users/zkoch/dream/data/tcga/processed_data/CPTAC-3_mutation.parquet


In [247]:
# correct number of case_ids, and sample_ids don't matter for mutation data (because mutations are defined as the exclusion of 2 samples)
cptac_mutation['first_sample_id'].value_counts()

first_sample_id
C3L-01913-31    22327
C3L-00586-31    13533
C3L-02557-31    12888
C3N-00151-03    12284
C3N-02637-31     9291
                ...  
C3L-01158-31        5
C3L-01052-31        4
C3N-02996-02        2
C3L-01253-71        1
C3L-00083-31        1
Name: count, Length: 1272, dtype: int64

# Expression

In [248]:
# no cases where first_sample_id cannot distinguish
sample_sheet.query("`Data Category` == 'Transcriptome Profiling'")[[ 'first_sample_id']].value_counts()

first_sample_id
1104806            1
C3N-01755-11       1
C3N-01719-04       1
C3N-01752-02       1
C3N-01754-03       1
                  ..
C3L-02701-02       1
C3L-02669-06       1
C3L-02669-03       1
C3L-02665-06       1
PT-Y8DK-01         1
Name: count, Length: 2220, dtype: int64

In [251]:
cptac_expr = process_tcgabiolinks.main(
    path = "/cellar/users/zkoch/dream/data/tcga/GDCdata/CPTAC-3/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification",
    dataset_name = 'CPTAC-3',
    output_path = "/cellar/users/zkoch/dream/data/tcga/processed_data",
    datatype = 'expression',
    sample_sheet_fn = "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample_sheet_no_duplicate_rows.tsv"
)

Read 2261 files
Saved to /cellar/users/zkoch/dream/data/tcga/processed_data/CPTAC-3_expression.parquet


In [252]:
cptac_expr.columns.value_counts()

C3N-00315-03    1
C3N-03488-03    1
C3N-02811-02    1
C3L-02179-01    1
C3L-00781-14    1
               ..
C3L-01281-06    1
C3N-02192-06    1
C3N-01715-02    1
C3N-00168-04    1
C3L-01302-03    1
Name: count, Length: 2220, dtype: int64

# Methylation

In [233]:
# no cases where first_case_id and first_sample_id cannot distinguish
sample_sheet.query("`Data Category` == 'DNA Methylation'")[['first_case_id', 'first_sample_id']].value_counts()

first_case_id  first_sample_id               
11LU013        9f905736-f662-41d6-b3ac-16758d    1
C3N-01645      C3N-01645-02                      1
C3N-01652      C3N-01652-05                      1
               C3N-01652-04                      1
C3N-01651      C3N-01651-10                      1
                                                ..
C3L-02741      C3L-02741-01                      1
C3L-02708      C3L-02708-02                      1
C3L-02707      C3L-02707-75                      1
C3L-02705      C3L-02705-71                      1
C464694        SM-JU33G                          1
Name: count, Length: 1858, dtype: int64

# Metadata

In [6]:
sample_sheet = pd.read_csv(
    "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample_sheet_no_duplicate_rows.tsv",
    sep = "\t"
    )
clinical = pd.read_csv(
    "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/clinical.tsv",
    sep = "\t"
    )

followup = pd.read_csv(
    "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/follow_up.tsv",
    sep = "\t"
    )
biospecimen_sample = pd.read_csv(
    "/cellar/users/zkoch/dream/data/tcga/manual_download/CPTAC-3/sample.tsv",
    sep = "\t"
    )

#### Get everything interesting from sample sheet

In [16]:
metadata = sample_sheet[
    ['first_case_id','first_sample_id','Data Type','Sample Type' ]
    ].copy(deep = True)
metadata.columns = ['case_id', 'sample_id', 'data_type', 'sample_type']
metadata['data_type'] = metadata['data_type'].map(
    {'Gene Expression Quantification': 'expression',
     'Masked Somatic Mutation': 'mutation', 
     'Methylation Beta Value': 'methylation'
     })
# all sample types are unique 
metadata['sample_type'] = metadata['sample_type'].str.split(", ", expand = True)[0]
# set all data tyope = mutation rows to have sample_type = 'Primary Tumor, Blood Derived Normal'
metadata.loc[metadata['data_type'] == 'mutation', 'sample_type'] = 'Primary Tumor, Blood Derived Normal'

In [40]:
metadata.groupby(['data_type'])[['sample_type']].value_counts()

data_type    sample_type                        
expression   Primary Tumor                          1567
             Solid Tissue Normal                     653
methylation  Primary Tumor                          1289
             Solid Tissue Normal                     568
             Recurrent Tumor                           1
mutation     Primary Tumor, Blood Derived Normal    1277
Name: count, dtype: int64

#### From biospecimen

In [52]:
biospecimen_sample.columns

Index(['project_id', 'case_id', 'case_submitter_id', 'sample_id',
       'sample_submitter_id', 'biospecimen_anatomic_site',
       'biospecimen_laterality', 'catalog_reference', 'composition',
       'current_weight', 'days_to_collection', 'days_to_sample_procurement',
       'diagnosis_pathologically_confirmed', 'distance_normal_to_tumor',
       'distributor_reference', 'freezing_method', 'growth_rate',
       'initial_weight', 'intermediate_dimension', 'is_ffpe',
       'longest_dimension', 'method_of_sample_procurement', 'oct_embedded',
       'passage_count', 'pathology_report_uuid', 'preservation_method',
       'sample_ordinal', 'sample_type', 'sample_type_id', 'shortest_dimension',
       'specimen_type', 'state', 'time_between_clamping_and_freezing',
       'time_between_excision_and_freezing', 'tissue_collection_type',
       'tissue_type', 'tumor_code', 'tumor_code_id', 'tumor_descriptor'],
      dtype='object')

#### From clinical

In [122]:
cols_we_want = ['case_submitter_id', 'days_to_birth', 'days_to_death','gender', 'race','vital_status','age_at_diagnosis' ,'primary_diagnosis','tissue_or_organ_of_origin']

In [123]:
metadata = metadata.merge(
    clinical[cols_we_want], left_on='case_id',
    right_on='case_submitter_id', how='left'
    )

#### From followup

In [139]:
# drop columns that are all '--
followup = followup.drop(
    columns = followup.columns[followup.apply(lambda x: x.str.contains('--').all())]
    )

In [164]:
len(set(metadata['case_id'].to_list()).intersection(set(followup['case_submitter_id'].to_list())))

1170

In [170]:
follow_up_cols_we_want = ['case_submitter_id', 'bmi','height', 'weight', 'comorbidity','days_to_recurrence', 'karnofsky_performance_status','ecog_performance_status']
metadata = metadata.merge(
    followup[follow_up_cols_we_want], left_on='case_id',
    right_on='case_submitter_id', how='left'
    )

#### General clean up and conversion

In [182]:
# replace '-- with np.nan
metadata = metadata.replace('\'--', np.nan)

In [195]:
# convert days_to_birth (a string) to years old (a float)
metadata['age_at_index'] = -1 * metadata['days_to_birth'].astype(float)/365.25

metadata['years_to_death'] = metadata['days_to_death'].astype(float)/365.25
metadata['years_to_recurrence'] = metadata['days_to_recurrence'].astype(float)/365.25
# drop stuff
metadata = metadata.drop(columns = ['days_to_birth', 'days_to_death', 'age_at_diagnosis'])
# rename columns
metadata = metadata.rename(columns = {'case_submitter_id': 'case_id'})

In [200]:
metadata.columns

Index(['case_id', 'sample_id', 'data_type', 'sample_type', 'gender', 'race',
       'vital_status', 'primary_diagnosis', 'tissue_or_organ_of_origin', 'bmi',
       'height', 'weight', 'comorbidity', 'karnofsky_performance_status',
       'ecog_performance_status', 'age_at_index', 'years_to_death',
       'years_to_recurrence'],
      dtype='object')

In [201]:
metadata = metadata[['case_id', 'sample_id', 'data_type', 'sample_type', 'age_at_index','tissue_or_organ_of_origin', 'gender', 'race',
       'vital_status', 'primary_diagnosis',  'bmi',
       'height', 'weight', 'comorbidity', 'karnofsky_performance_status',
       'ecog_performance_status', 'years_to_death',
       'years_to_recurrence'
        ]]
metadata.reset_index(drop = True, inplace = True)

In [207]:
metadata.to_parquet("/cellar/users/zkoch/dream/data/tcga/processed_data/CPTAC-3_metadata.parquet")